## Basic plotting examples with matplolib, seaborn, bokeh and holoviews
Few plotting examples using DR1 data

In [ ]:
%matplotlib inline
import easyaccess as ea
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
print('easyaccess: {0}'.format(ea.__version__))

In [ ]:
# Connecting to the DES DB, by default is connecting to the public DB
connection = ea.connect()

## Running a simple query into pandas

In [ ]:
query = """
--
-- Example Query --
SELECT
  COADD_OBJECT_ID, RA, DEC, MAG_AUTO_I, MAG_AUTO_R, MAG_AUTO_Y, TILENAME 
FROM 
  DR1_MAIN_SAMPLE sample (1)
WHERE
  MAG_AUTO_R between 17 and 30 and  MAG_AUTO_I between 17 and 30
"""

In [ ]:
df = connection.query_to_pandas(query)

In [ ]:
# Print the tail of the table
df.tail()

### Matplotlib plots

In [ ]:
# do some plots
X, Y = np.mgrid[17:27:70j, 17:27:70j]
positions = np.vstack([X.ravel(), Y.ravel()])
values = np.vstack([df.MAG_AUTO_R, df.MAG_AUTO_I])
kernel = stats.gaussian_kde(values)
Z = np.reshape(kernel(positions).T, X.shape)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(np.rot90(Z), cmap=plt.cm.gist_earth_r,extent=[17, 27, 17, 27])
plt.xlabel('MAG_AUTO_R')
plt.ylabel('MAG_AUTO_I')

### Example using seaborn

In [ ]:
plt.figure(figsize=(10,10))
sns.set_style("whitegrid")
temp=sns.JointGrid(df.MAG_AUTO_R, df.MAG_AUTO_I,xlim=[17,27],ylim=[17,27],space=0, size=6)
temp.plot_joint(sns.kdeplot,cmap='jet',alpha=0.8,n_levels=10)
temp.plot_joint(plt.scatter, s= .3,color='black')
temp.plot_marginals(sns.distplot, color='green',kde=True)

### Some interactive plots using Bokeh and Holoviews

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
hextiles = hv.HexTiles(df, [('MAG_AUTO_R', 'R'), ('MAG_AUTO_I', 'I')], [], extents=(20,26,20,26))

In [ ]:
hextiles.options(width=500, height=500, min_count=0, tools=['hover'], colorbar=True, ) * hv.Bivariate(hextiles).options(show_legend=False)

### More interactive plots linked together

In [ ]:
import param
from holoviews import streams
hv.extension('bokeh')

In [ ]:
# Transform data across RA =0 
df['RA'][df['RA']>180] -= 360

By creating a HoloViews dataset we annotate the dataset for visualization declaring which columns correspond to independent variables (called key dimensions or kdims) and dependent variables (called value dimensions or vdims).

In [ ]:

kdims = [('RA', 'RA(deg)'), ('DEC', 'Dec(deg)')]
vdims = [('MAG_AUTO_R', 'R')]
ds = hv.Dataset(df, kdims, vdims)
ds

### Let's plot the coordinates

In [ ]:
%%opts Points  [height=450 width=450 tools=['hover', 'box_select']]
points = hv.Points(ds).options({'Points': {'size': 1, 'alpha': 0.5}})
points

### Let's plot the histogram for MAG_R

In [ ]:
%%opts Histogram  [height=500 width=500] (fill_color='red', alpha=0.4)
frequencies, edges = np.histogram(ds.data['MAG_AUTO_R'],bins=30)
hist = hv.Histogram((frequencies, edges),vdims='# Number',kdims='MAG_R')
hist

### Let's make a toy linked plot

In [ ]:
# We need a function for every time there is a stream (from a selection box)
def update_histogram(index):
    # this will initialize the histogram with data in case there's no selection
    if len(index) > 0:
        selected_flux = ds.data.iloc[index]['MAG_AUTO_R']
    else:
        selected_flux = ds.data['MAG_AUTO_R']
    
    frequencies, edges = np.histogram(selected_flux, bins=24, normed=True)
    
    hist = hv.Histogram((frequencies, edges), vdims='Normalized',kdims='MAG_R')
    return hist

### As before we define the points and the histogram (hv.Histogram) which is dynamically generated

In [ ]:
selection = streams.Selection1D(source=points)
print('The %s stream has contents %r' % (selection, selection.contents))

In [ ]:
%%opts Histogram  [height=450 width=450] (fill_color='red', alpha=0.4)
dmap = hv.DynamicMap(update_histogram, streams=[selection])
points + dmap